In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from sklearn.datasets import load_iris
import pandas as pd

content = load_iris()

data = pd.DataFrame(content.data, columns=[f"x{i+1}" for i in range(content.data.shape[1])])
data["y"] = (content.target == 1).astype(int)

In [2]:
content.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [58]:
from estyp.linear_model.stepwise import forward_selection, both_selection
from estyp.linear_model import LogisticRegression

ff1 = forward_selection(
    y       = "y",
    data    = data,
    model   = LogisticRegression,
    verbose = False,
)
ff2 = both_selection(
    formula = formula1,
    data    = data,
    model   = LogisticRegression,
    verbose = False
)
print("- Forward result:", ff1)
print("- Both result   :", ff2)

- Forward result: y ~ x2
- Both result   : y ~ x1 + x2 + x3 + x4


In [60]:
from estyp.testing import nested_models_test

model1 = LogisticRegression.from_formula(ff1, data).fit()
model2 = LogisticRegression.from_formula(ff2, data).fit()

nested_models_test(model1, model2) # First model is nested in the second one


    Nested models F-test
    F = 2.2843 | df: {'df_num': 3, 'df_den': 145} | p-value = 0.0814
    alternative hypothesis: big model is true
    sample estimates:
      Difference in deviances between models: 6.856231
    

With $\alpha=0.05$, the null hypothesis is not rejected: model2 is significantly not better than model1.